In [75]:
import re
import numpy as np

import polars as pl
import csv

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [36]:
file_paths = {
    "test_arabic_negative_tweets": "../data/test_arabic_negative_tweets.tsv",
    "test_arabic_positive_tweets": "../data/test_arabic_positive_tweets.tsv",
    "train_arabic_negative_tweets": "../data/train_arabic_negative_tweets.tsv",
    "train_arabic_positive_tweets": "../data/train_arabic_positive_tweets.tsv",
}

In [41]:
def load_data(file_path: str, label: int) -> pl.DataFrame:
    """
    Load data from a TSV file and assign a label to each row.

    Args:
        file_path (str): The path to the TSV file.
        label (int): The label to assign to each tweet (0 for negative, 1 for positive).

    Returns:
        pl.DataFrame: A Polars DataFrame containing tweets and their labels.
    """
    rows = []
    with open(file_path, newline="", encoding="utf-8") as tsvfile:
        reader = csv.reader(tsvfile, delimiter="\t")
        for row in reader:
            rows.append([row[1], label])
    return pl.DataFrame(rows, schema=["tweet", "label"])


def preprocess_text(text: str) -> str:
    """
    Preprocess a tweet by removing mentions, URLs, punctuation, and extra spaces.

    Args:
        text (str): The tweet text to preprocess.

    Returns:
        str: The cleaned tweet text.
    """
    text = re.sub(r"@\w+", "", text)  # Remove mentions
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

In [84]:
train_neg = load_data(file_paths["train_arabic_negative_tweets"], 0)
train_pos = load_data(file_paths["train_arabic_positive_tweets"], 1)
test_neg = load_data(file_paths["test_arabic_negative_tweets"], 0)
test_pos = load_data(file_paths["test_arabic_positive_tweets"], 1)

train_df = pl.concat([train_neg, train_pos])
test_df = pl.concat([test_neg, test_pos])
df = pl.concat([train_df, test_df])
df = df.sample(fraction=1, shuffle=True)
df.head()

tweet,label
str,i64
"""جديد بطلة اسطنبول الظالمه 💕 #z…",1
"""الحين صار كل شي عندنا الكتروني…",0
"""✨ إعلام هايدي وتبعه الأصفر يحا…",1
"""الي راح حمادة والي جاي حمادة ت…",1
"""لك في صدي مملكة حب وغرام 💙""",1


In [85]:
df = df.with_columns(
    [
        pl.col("tweet")
        .map_elements(preprocess_text, return_dtype=str)
        .alias("cleaned_tweet")
    ]
)

In [86]:
X = df["cleaned_tweet"].to_numpy()
y = df["label"].to_numpy()

vectorizer = CountVectorizer()
X_vectorized = vectorizer.fit_transform(X)

In [87]:
vectorizer.vocabulary_

{'جديد': 32202,
 'بطلة': 24447,
 'اسطنبول': 7510,
 'الظالمه': 14675,
 'zalimistanbul': 1126,
 'الحين': 12364,
 'صار': 41058,
 'كل': 48955,
 'شي': 40882,
 'عندنا': 44346,
 'الكتروني': 16111,
 'طيب': 42624,
 'فيه': 47135,
 'كبار': 48459,
 'في': 46978,
 'السن': 13703,
 'أو': 4150,
 'ناس': 59828,
 'ما': 53423,
 'تفهم': 30281,
 'بالتقنية': 21237,
 'كثير': 48596,
 'حلول': 34097,
 'لهم': 52943,
 'ولا': 71308,
 'لا': 49689,
 'إعلام': 4747,
 'هايدي': 62172,
 'وتبعه': 67319,
 'الأصفر': 9236,
 'يحاضرون': 75849,
 'عن': 44291,
 'الفضيلة': 15572,
 'والشرف': 65663,
 'ويوسمون': 73939,
 'الأسطورةأبوعمر': 9190,
 'بأقذع': 20573,
 'العبارات': 14797,
 'ويتصدرون': 73209,
 'المنابر': 17903,
 'ويت': 73187,
 'الي': 18997,
 'راح': 36693,
 'حمادة': 34126,
 'والي': 66454,
 'جاي': 32013,
 'تاني': 27053,
 'خالص': 34595,
 'الناس': 18244,
 'طلعت': 42487,
 'واحتفلت': 64130,
 'دي': 36189,
 'عارفة': 42877,
 'أن': 3894,
 'الشغل': 14019,
 'الجد': 11702,
 'يادوب': 74441,
 'حيبدأ': 34395,
 'اسقاط': 7546,
 'النظام': 18425,
 

In [88]:
tfidf = TfidfTransformer(use_idf=True, norm="l2", smooth_idf=True)
X_tfidf = tfidf.fit_transform(vectorizer.fit_transform(X)).toarray()

In [90]:
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y, test_size=0.2, stratify=y
)

In [91]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [92]:
y_pred = model.predict(X_test)

In [93]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

In [94]:
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.7814948498987587
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.81      0.79      5656
           1       0.80      0.75      0.78      5703

    accuracy                           0.78     11359
   macro avg       0.78      0.78      0.78     11359
weighted avg       0.78      0.78      0.78     11359

